In [1]:
import pandas as pd
import numpy as np
import json
import os, sys
import matplotlib.pyplot as plt
current_directory = os.getcwd()
sys.path.append(f'{current_directory}/../src/')

from transformers import AutoTokenizer
import keras
import helpers as hp


Using TensorFlow backend.


## 1. Load Data

In [2]:
# Train-Valid data
data = json.load(open(f'{current_directory}/../data/train.json'))

#labels
labels = ["B-EMAIL", "B-ID_NUM", "B-NAME_STUDENT", "B-PHONE_NUM",
        "B-STREET_ADDRESS", "B-URL_PERSONAL", "B-USERNAME",
        "I-ID_NUM", "I-NAME_STUDENT", "I-PHONE_NUM",
        "I-STREET_ADDRESS","I-URL_PERSONAL","O"]

help = hp.helpers(labels,data)

In [3]:
#Convert labels to ids
data = help.labels_to_ids()

## 2. Plot Labels

In [ ]:
# Get unique labels and their frequency
all_labels = np.array([x for label in labels for x in label])
unique_labels, label_counts = np.unique(all_labels, return_counts=True)
labels_plt = [id2label[i] for i in unique_labels]

def addlabels(x,y):
    for i in range(len(x)):
        plt.text(i, y[i], y[i], ha = 'center')
        
plt.figure(figsize =(25, 10))
plt.bar(labels_plt,label_counts)
addlabels(labels_plt, label_counts)
plt.yscale("log")
plt.show()

## 3. Preprocessing

### 3.1 Tokenization

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

In [6]:
example = data[0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens[0:10])
print(tokenized_input["input_ids"][0:10])
print(tokenized_input.word_ids(batch_index=0)[0:10])

['[CLS]', 'design', 'thinking', 'for', 'innovation', 'reflex', '##ion', '-', 'av', '##ril']
[101, 2640, 3241, 2005, 8144, 22259, 3258, 1011, 20704, 15928]
[None, 0, 1, 2, 3, 4, 4, 5, 6, 6]


In [7]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], is_split_into_words=True)

    labels = []
    label=examples[f"labels_id"]
    word_ids = tokenized_inputs.word_ids()  # Map tokens to their respective word.
    previous_word_idx = None
    label_ids = []
    for word_idx in word_ids:  # Set the special tokens to -100.
        if word_idx is None:
            label_ids.append(-100)
        elif word_idx != previous_word_idx:  # Only label the first token of a given word.
            label_ids.append(label[word_idx])
        else:
            label_ids.append(-100)
        previous_word_idx = word_idx

    tokenized_inputs["labels_ids"] = label_ids
    tokenized_inputs["document"] = examples["document"]
    return tokenized_inputs

In [8]:
l = tokenize_and_align_labels(data[0])

In [9]:
print(tokens[0:20])
print(l["input_ids"][20:0])
print(l.word_ids(batch_index=0)[0:20])
print(l["labels_ids"][0:20])
print(l["document"])

['[CLS]', 'design', 'thinking', 'for', 'innovation', 'reflex', '##ion', '-', 'av', '##ril', '2021', '-', 'nat', '##hal', '##ie', 'sy', '##lla', 'challenge', '&', 'selection']
[]
[None, 0, 1, 2, 3, 4, 4, 5, 6, 6, 7, 8, 9, 9, 9, 10, 10, 12, 13, 14]
[-100, 12, 12, 12, 12, 12, -100, 12, 12, -100, 12, 12, 2, -100, -100, 8, -100, 12, 12, 12]
7


In [11]:
tokenized_data = help.tokenized_data(tokenizer)

TypeError: 'list' object is not callable

In [12]:
_tokenized_data = []
for _example in help.data:
    _tokenized_data.append(tokenize_and_align_labels(_example,tokenizer))

TypeError: tokenize_and_align_labels() takes 1 positional argument but 2 were given

In [15]:
help.tokenize_and_align_labels(data[0],tokenizer)

NameError: name 'tokenized_inputs' is not defined